## 1.1 Load

In [1]:

import pandas as pd
from function import *


abandon_2d_variable = [
    "current_abandonment",
    "recultivation", 
    "abandonment_duration",
    "abandonment_year"
]
fea_3d_variable = [
    'GDPpc',
    'GDPtot',
    'GURdist',
    'Population',
    'gdmp',
    'rsds',
    'tas',
    'wind'
]
fea_2d_variable = [
    'DEM',
    'Powerdist',
    'PrimaryRoad',
    'SecondaryRoad',
    'Slope',
    'TertiaryRoad'
]
PATHS = {
    'abandonment': r"D:\xarray\merged_chunk_2\*.nc",
    'feature':     "D:/xarray/aligned2/Feature_all/*.nc",
    'csv':         "data/aligned_for_training0519.csv",
}

YEARS = [2018, 2020]
time=['2018-01-01','2020-01-01']
# 2. 读取 PV 站点并切片（测试模式）
pv_df = load_pv_sites(PATHS['csv'], years=YEARS)
# Convert lon and lat columns to float32

pv_df['lon'] = pv_df['lon'].astype('float32')
pv_df['lat'] = pv_df['lat'].astype('float32')

# Rename 'year' to 'time' and convert to datexxtime64
pv_df = pv_df.rename(columns={'year': 'time'})
pv_df['time'] = pd.to_datetime(pv_df['time'], format='%Y')


# 1. 打开并 rechunk
ds_abandon, ds_feat = load_datasets(
    PATHS['abandonment'], PATHS['feature']
)



ds_merge=xr.merge([ds_abandon, ds_feat])
# Convert coordinates to float32 while preserving other variables
ds_merge = ds_merge.assign_coords({
    'lon': ds_merge.lon.astype('float32'),
    'lat': ds_merge.lat.astype('float32')
})

# For variables without time dimension, expand them to have same value for all times
for var in ds_merge.data_vars:
    if 'time' not in ds_merge[var].dims:
        # Expand the variable to have time dimension with same values
        ds_merge[var] = ds_merge[var].expand_dims(time=ds_merge.time)
ds_merge

c:\Users\zpy10\Anaconda3\envs\geo\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


gogogo


c:\Users\zpy10\Anaconda3\envs\geo\Lib\site-packages\dask\array\core.py:4998: PerformanceWarning: Increasing number of chunks by factor of 43
  result = blockwise(
c:\Users\zpy10\Anaconda3\envs\geo\Lib\site-packages\dask\array\core.py:4998: PerformanceWarning: Increasing number of chunks by factor of 43
  result = blockwise(
c:\Users\zpy10\Anaconda3\envs\geo\Lib\site-packages\dask\array\core.py:4998: PerformanceWarning: Increasing number of chunks by factor of 43
  result = blockwise(
c:\Users\zpy10\Anaconda3\envs\geo\Lib\site-packages\dask\array\core.py:4998: PerformanceWarning: Increasing number of chunks by factor of 43
  result = blockwise(
c:\Users\zpy10\Anaconda3\envs\geo\Lib\site-packages\dask\array\core.py:4998: PerformanceWarning: Increasing number of chunks by factor of 174
  result = blockwise(
c:\Users\zpy10\Anaconda3\envs\geo\Lib\site-packages\dask\array\core.py:4998: PerformanceWarning: Increasing number of chunks by factor of 14
  result = blockwise(
c:\Users\zpy10\Anacon

<xarray.Dataset> Size: 3TB
Dimensions:               (time: 31, lat: 21600, lon: 43200)
Coordinates:
  * time                  (time) datetime64[ns] 248B 1992-01-01 ... 2022-01-01
    spatial_ref           int32 4B 0
    band                  int32 4B 1
  * lon                   (lon) float32 173kB -180.0 -180.0 ... 180.0 180.0
  * lat                   (lat) float32 86kB -90.0 -89.99 -89.98 ... 89.99 90.0
Data variables: (12/19)
    abandonment_year      (time, lat, lon) float32 116GB dask.array<chunksize=(31, 500, 496), meta=np.ndarray>
    abandonment_duration  (time, lat, lon) float32 116GB dask.array<chunksize=(31, 500, 496), meta=np.ndarray>
    recultivation         (time, lat, lon) float32 116GB dask.array<chunksize=(31, 500, 496), meta=np.ndarray>
    current_abandonment   (time, lat, lon) float32 116GB dask.array<chunksize=(31, 500, 496), meta=np.ndarray>
    landcover             (time, lat, lon) float32 116GB dask.array<chunksize=(16, 250, 249), meta=np.ndarray>
    DEM                   (time, lat, lon) float32 116GB dask.array<chunksize=(31, 1440, 2880), meta=np.ndarray>
    ...                    ...
    rsds                  (time, lat, lon) float32 116GB dask.array<chunksize=(1, 3085, 6172), meta=np.ndarray>
    SecondaryRoad         (time, lat, lon) float64 231GB dask.array<chunksize=(31, 981, 1964), meta=np.ndarray>
    Slope                 (time, lat, lon) float32 116GB dask.array<chunksize=(31, 1440, 2880), meta=np.ndarray>
    tas                   (time, lat, lon) float64 231GB dask.array<chunksize=(1, 2700, 5400), meta=np.ndarray>
    TertiaryRoad          (time, lat, lon) float64 231GB dask.array<chunksize=(31, 981, 1964), meta=np.ndarray>
    wind                  (time, lat, lon) float32 116GB dask.array<chunksize=(1, 432, 500), meta=np.ndarray>

## 1.2 ZZZ

In [2]:
pv_df
pv_df['year'] = pd.to_datetime(pv_df['time']).dt.year
pv_df_dedup = (
    pv_df.sort_values('time')  # 可改为其他排序依据
         .drop_duplicates(subset=['year', 'lat', 'lon'], keep='last')  # 你也可以用 keep='first'
         .reset_index(drop=True)
)
pv_df_dedup

,unique_id,p_area,capacity_m,country,time,lon,lat,year
0,1,13592.361840,1.201704,GRC,2018-01-01,21.612499,38.112499,2018
1,45763,3005.453465,0.281647,CHN,2018-01-01,116.545830,33.420834,2018
2,45764,2134.049502,0.200225,CHN,2018-01-01,116.520836,32.987499,2018
3,45765,3429.546491,0.321102,CHN,2018-01-01,116.504166,34.495834,2018
4,45766,2231.352132,0.209629,CHN,2018-01-01,116.562500,32.287498,2018
...,...,...,...,...,...,...,...,...
90825,11758,40069.483340,2.684475,USA,2020-01-01,-78.079170,35.312500,2020
90826,11759,56.397704,0.000000,USA,2020-01-01,-79.037498,36.004166,2020
90827,11760,22.106298,0.000000,USA,2020-01-01,-79.054169,35.954166,2020
90828,11754,26182.946620,1.875714,USA,2020-01-01,-78.262497,36.420834,2020


## 1.3 SDFD

In [12]:
from tqdm.auto import tqdm
import numpy as np

def process_chunk(df, ds_merge, step,stop=0):

    # Get coordinates that exist in both df_temp and pv_df
    common_lats = np.intersect1d(ds_merge.lat.values, df['lat'].unique())
    common_lons = np.intersect1d(ds_merge.lon.values, df['lon'].unique())

    # Select data from df_temp using only the common coordinates
    ds_merge = ds_merge.sel(
        lat=common_lats,
        lon=common_lons,
    )

    #step = 500
    total_lat = len(ds_merge.lat)
    total_lon = len(ds_merge.lon)
    merged_dfs = []
    
    # Calculate total iterations for progress bar
    total_iterations = (total_lat // step + (1 if total_lat % step else 0)) * \
                    (total_lon // step + (1 if total_lon % step else 0))

    # Create progress bar with Chinese description
    pbar = tqdm(total=total_iterations, desc="处理数据块")

    # Iterate through all latitude and longitude points in chunks
    for start_lat in range(0, total_lat, step):
        end_lat = min(start_lat + step, total_lat)
        for start_lon in range(0, total_lon, step):
            end_lon = min(start_lon + step, total_lon)
            #print(1)
            # Extract data from ds_merge for each time point
            df = ds_merge.isel(
                lat=slice(start_lat, end_lat),
                lon=slice(start_lon, end_lon)
            ).compute().to_dataframe()
            
            # Reset index to convert multi-index to columns
            df = df.reset_index()
            
            # Merge with pv_df based on lat/lon coordinates
            chunk_merged = pd.merge(df, pv_df_dedup, on=['lat','lon','time'], how='inner')
            merged_dfs.append(chunk_merged)
            
            # Update progress bar
            pbar.update(1)
            if stop==1:
                break
    # Close progress bar
    pbar.close()
    # Combine all chunks into final dataframe
    merged_df = pd.concat(merged_dfs, ignore_index=True)
    return merged_df


# Iterate through each variable
# Get all variables from ds_merge
all_vars = list(ds_merge.data_vars)
merged_dfs = []
# Iterate through each variable
for i, var in enumerate(all_vars):
    print(f"Processing variable {i+1} of {len(all_vars)}: {var}")
    # Check if variable has time dimension
    df_temp = ds_merge[var].sel(time=['2018-01-01','2020-01-01'])

    # Process the chunk and merge with pv_df
    merged_df = process_chunk(pv_df_dedup, df_temp, step=3000)

    merged_dfs.append(merged_df)

# Concatenate all merged dataframes with single-column retention

final_merged_df = merged_dfs[0]
for df in tqdm(merged_dfs[1:], desc="合并数据"):

    final_merged_df = pd.merge(
        final_merged_df,
        df,
        on=["time", "lon", "lat"],
        how="inner",
        suffixes=("", "_drop")
    )
    # Drop duplicated columns with "_drop" suffix
    final_merged_df = final_merged_df.loc[:, ~final_merged_df.columns.str.endswith("_drop")]



Processing variable 1 of 19: abandonment_year


处理数据块:   0%|          | 0/18 [00:00<?, ?it/s]

Processing variable 2 of 19: abandonment_duration


处理数据块:   0%|          | 0/18 [00:00<?, ?it/s]

Processing variable 3 of 19: recultivation


...

Processing variable 18 of 19: tas


处理数据块:   0%|          | 0/18 [00:00<?, ?it/s]

Processing variable 19 of 19: wind


处理数据块:   0%|          | 0/18 [00:00<?, ?it/s]

合并数据:   0%|          | 0/18 [00:00<?, ?it/s]

In [4]:
import dask.dataframe as dd

# 将每个 pandas.DataFrame 转为 Dask DataFrame（默认划分成 4 个分区）
ddfs = [dd.from_pandas(df, npartitions=4) for df in merged_dfs]

# 初始化为第一个
merged_ddf = ddfs[0]

# 逐个合并后续 DataFrames
for i, ddf in enumerate(ddfs[1:], start=1):
    print(f"Merging variable {i+1} of {len(ddfs)}")
    merged_ddf = merged_ddf.merge(ddf, on=["time", "lon", "lat"], how="inner", suffixes=("", "_drop"))
    # 删除带 "_drop" 后缀的重复列
    merged_ddf = merged_ddf.loc[:, [col for col in merged_ddf.columns if not col.endswith("_drop")]]


output_path = "D:/xarray/aligned2/temp/final_merged_*.csv"
merged_ddf.to_csv(output_path, index=False, single_file=False)
print("✅ Final Dask-merged CSVs saved.")


Merging variable 2 of 6
Merging variable 3 of 6
Merging variable 4 of 6
Merging variable 5 of 6
Merging variable 6 of 6
✅ Final Dask-merged CSVs saved.


In [ ]:
import os
import pandas as pd

def write_each_variable_parquet(merged_dfs, save_dir, base_cols=["time", "lat", "lon"]):
    os.makedirs(save_dir, exist_ok=True)
    for i, df in enumerate(merged_dfs):
        var_cols = [c for c in df.columns if c not in base_cols and c not in ['band', 'spatial_ref']]
        if len(var_cols) != 1:
            print(f"❌ Warning: variable {i} has unexpected columns: {var_cols}")
            continue
        var_name = var_cols[0]
        out_df = df[base_cols + [var_name]]
        out_path = os.path.join(save_dir, f"{var_name}.parquet")
        out_df.to_parquet(out_path, index=False)
        print(f"✅ Saved {var_name}.parquet")


import duckdb
import glob

def join_all_parquet_to_csv(parquet_dir, output_csv):
    files = sorted(glob.glob(f"{parquet_dir}/*.parquet"))
    
    # 构造 DuckDB SQL JOIN 查询
    base = f"read_parquet('{files[0]}')"
    for f in files[1:]:
        base = f"({base}) JOIN read_parquet('{f}') USING (time, lat, lon)"
    
    # 执行 SQL，并写出 CSV（或 Parquet）
    con = duckdb.connect()
    con.execute(f"COPY ({base}) TO '{output_csv}' (HEADER, DELIMITER ',')")
    con.close()
    print(f"🎯 Final merged training table saved at: {output_csv}")


# 第一步：保存每个变量
write_each_variable_parquet(merged_dfs, save_dir="D:/xarray/aligned2/vars")

# 第二步：用 DuckDB 整合为训练表
join_all_parquet_to_csv(
    parquet_dir="D:/xarray/aligned2/vars",
    output_csv="D:/xarray/aligned2/final_training.csv"
)


ModuleNotFoundError: No module named 'duckdb'

In [ ]:
save_dir=r"D:\xarray\aligned2\temp_dfs"
all_files = sorted([f for f in os.listdir(save_dir) if f.endswith(".parquet") and f.startswith("df_")])
unmerge_df=[]



for i, file in enumerate(all_files):
    df_path = os.path.join(save_dir, file)
    df=pd.read_parquet(df_path)
    unmerge_df.append(df)

    if i==5:
        break


len(unmerge_df)
unmerge_df[1]



,time,lat,lon,spatial_ref,band,abandonment_duration,unique_id,p_area,capacity_m,country
0,2018-01-01,-13.837500,-171.804169,0,1,NaN,26510,87614.296875,7.850205,WSM
1,2018-01-01,-13.829166,-171.995834,0,1,NaN,26500,10696.658203,0.956408,WSM
2,2018-01-01,-13.829166,-171.987503,0,1,NaN,26501,66982.132812,5.989546,WSM
3,2020-01-01,-29.245832,-177.929169,0,1,NaN,19077,385.150726,0.000000,NZL
4,2020-01-01,-22.645834,-152.795837,0,1,NaN,34745,615.614868,0.000000,PYF
...,...,...,...,...,...,...,...,...,...,...
103912,2020-01-01,55.712502,37.462502,0,1,NaN,32467,75.500473,0.000000,RUS
103913,2020-01-01,55.737499,37.579166,0,1,NaN,9832,281.253357,0.000000,RUS
103914,2020-01-01,55.820835,37.704166,0,1,NaN,32288,44.549313,0.000000,RUS
103915,2020-01-01,55.979168,37.262501,0,1,NaN,17828,538.808228,0.000000,RUS


In [13]:
final_merged_df

,time,lat,lon,spatial_ref,band,abandonment_year,unique_id,p_area,capacity_m,country,...,Population,Powerdist,PrimaryRoad,SecondaryRoad,Slope,TertiaryRoad,gdmp,rsds,tas,wind
0,2018-01-01,-13.837500,-171.804169,0,1,NaN,26510,87614.298290,7.850205,WSM,...,646.119934,622589.0000,12.719680,0.000000,2.521324,197.039190,NaN,19.063463,2995.384528,2.293710
1,2018-01-01,-13.829166,-171.995834,0,1,NaN,26500,10696.657940,0.956408,WSM,...,138.296951,622588.9375,14.276298,0.000000,0.715136,100.218651,NaN,18.820724,2999.923652,2.770978
2,2018-01-01,-13.829166,-171.987503,0,1,NaN,26501,66982.132520,5.989546,WSM,...,138.296951,622588.9375,14.276298,0.000000,0.922556,100.218651,NaN,18.825073,2999.838979,2.721136
3,2018-01-01,-4.387500,-79.812500,0,1,NaN,10893,16680.699130,1.511097,ECU,...,15.645079,622609.3750,18.676649,24.508936,6.674829,43.411486,167.423999,18.745583,2969.209102,0.836408
4,2018-01-01,-4.254167,-79.470833,0,1,NaN,10880,34065.381000,3.038932,ECU,...,37.699821,622609.3750,6.512673,18.597032,7.650049,89.491292,128.872533,17.977419,2920.696152,3.776348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90816,2020-01-01,55.712502,37.462502,0,1,NaN,32467,75.500476,0.000000,RUS,...,14468.128906,622589.8125,62.451511,112.277082,1.968263,861.205086,NaN,9.967303,2804.318214,2.684799
90817,2020-01-01,55.737499,37.579166,0,1,NaN,9832,281.253369,0.000000,RUS,...,12943.362305,622589.8125,118.298899,140.942360,0.393874,1132.096021,NaN,9.992375,2804.168017,2.531498
90818,2020-01-01,55.820835,37.704166,0,1,NaN,32288,44.549314,0.000000,RUS,...,16157.847656,622589.8125,38.558650,73.424121,0.412605,642.344304,NaN,9.950527,2803.408293,2.519807
90819,2020-01-01,55.979168,37.262501,0,1,NaN,17828,538.808249,0.000000,RUS,...,4346.128418,622589.5000,63.311948,5.627337,0.978244,316.949814,NaN,9.815846,2800.570781,2.697752


## 1.4 Save

In [15]:
 # Remove spatial_ref and band columns
final_merged_df = final_merged_df.drop(['spatial_ref', 'band'], axis=1)

# Save to CSV
final_merged_df.to_csv('training_embedding.csv', index=False)




KeyboardInterrupt



备注：这里经过了年内去重操作、

In [18]:
final_merged_df

,time,lat,lon,abandonment_year,unique_id,p_area,capacity_m,country,year,abandonment_duration,...,Population,Powerdist,PrimaryRoad,SecondaryRoad,Slope,TertiaryRoad,gdmp,rsds,tas,wind
0,2018-01-01,-13.837500,-171.804169,NaN,26510,87614.298290,7.850205,WSM,2018,NaN,...,646.119934,622589.0000,12.719680,0.000000,2.521324,197.039190,NaN,19.063463,2995.384528,2.293710
1,2018-01-01,-13.829166,-171.995834,NaN,26500,10696.657940,0.956408,WSM,2018,NaN,...,138.296951,622588.9375,14.276298,0.000000,0.715136,100.218651,NaN,18.820724,2999.923652,2.770978
2,2018-01-01,-13.829166,-171.987503,NaN,26501,66982.132520,5.989546,WSM,2018,NaN,...,138.296951,622588.9375,14.276298,0.000000,0.922556,100.218651,NaN,18.825073,2999.838979,2.721136
3,2018-01-01,-4.387500,-79.812500,NaN,10893,16680.699130,1.511097,ECU,2018,NaN,...,15.645079,622609.3750,18.676649,24.508936,6.674829,43.411486,167.423999,18.745583,2969.209102,0.836408
4,2018-01-01,-4.254167,-79.470833,NaN,10880,34065.381000,3.038932,ECU,2018,NaN,...,37.699821,622609.3750,6.512673,18.597032,7.650049,89.491292,128.872533,17.977419,2920.696152,3.776348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90816,2020-01-01,55.712502,37.462502,NaN,32467,75.500476,0.000000,RUS,2020,NaN,...,14468.128906,622589.8125,62.451511,112.277082,1.968263,861.205086,NaN,9.967303,2804.318214,2.684799
90817,2020-01-01,55.737499,37.579166,NaN,9832,281.253369,0.000000,RUS,2020,NaN,...,12943.362305,622589.8125,118.298899,140.942360,0.393874,1132.096021,NaN,9.992375,2804.168017,2.531498
90818,2020-01-01,55.820835,37.704166,NaN,32288,44.549314,0.000000,RUS,2020,NaN,...,16157.847656,622589.8125,38.558650,73.424121,0.412605,642.344304,NaN,9.950527,2803.408293,2.519807
90819,2020-01-01,55.979168,37.262501,NaN,17828,538.808249,0.000000,RUS,2020,NaN,...,4346.128418,622589.5000,63.311948,5.627337,0.978244,316.949814,NaN,9.815846,2800.570781,2.697752


In [16]:
missing_keys = pd.merge(pv_df_dedup, final_merged_df, on=['lat', 'lon','time'], how='left', indicator=True)
missing = missing_keys[missing_keys['_merge'] == 'left_only']
missing

,unique_id_x,p_area_x,capacity_m_x,country_x,time,lon,lat,year_x,abandonment_year,unique_id_y,...,Powerdist,PrimaryRoad,SecondaryRoad,Slope,TertiaryRoad,gdmp,rsds,tas,wind,_merge
18639,67752,8082.070784,0.731052,GBR,2018-01-01,0.004167,53.487499,2018,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
18677,67834,2716.929277,0.245711,GBR,2018-01-01,0.004167,53.412498,2018,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
18883,68009,2457.466166,0.222334,GBR,2018-01-01,0.004167,53.437500,2018,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
20724,65832,426831.056900,31.896933,GBR,2018-01-01,0.004167,52.062500,2018,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
26806,56002,20156.432690,1.762364,ESP,2018-01-01,0.004167,38.812500,2018,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
73947,5807,4218.800539,0.000000,GBR,2020-01-01,0.004167,53.362499,2020,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
75772,5171,5906.320755,0.000000,ESP,2020-01-01,0.004167,38.812500,2020,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
77505,7497,562.506739,0.000000,GBR,2020-01-01,0.004167,52.062500,2020,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
85144,17155,345.945594,0.000000,GBR,2020-01-01,0.004167,52.079166,2020,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


# Load

In [ ]:
# float32, 